# Homework 4

## Problem 2: Simple unit commitment

Adapt the unit commitment code from class to model PHS as a battery. The model for that is the following:


$$
\begin{align}
\min \sum_{g \notin PHS, t \in T} VarCost_g \times GEN_{g,t} + \sum_{g \in G_{thermal}, t \in T} StartUpCost_g \times START_{g,t} + \\
\sum_{t \in T} VarCost_{PHS} \times (DISCHARGE_t + CHARGE_t) \\
\end{align}
$$
$$
\begin{align}
\text{s.t.} & \\
 & \sum_{g \notin PHS} GEN_{g,t} + DISCHARGE_t - CHARGE_t = Demand_t \quad \forall \quad t \in T\\
 & GEN_{g,t} \leq Pmax_{g,t} & \forall \quad g \notin G_{thermal} , t \in T \\
 & GEN_{g,t} \geq Pmin_{g,t} & \forall \quad g \notin G_{thermal} , t \in T \\
 & GEN_{g,t} \leq Pmax_{g,t} \times COMMIT_{g,t} & \forall \quad g \in G_{thermal} , t \in T \\
 & GEN_{g,t} \geq Pmin_{g,t} \times COMMIT_{g,t} & \forall \quad g \in G_{thermal} , t \in T \\
 & COMMIT_{g,t} \geq \sum_{t'≥t-MinUp_g}^{t} START_{g,t} & \forall \quad g \in G_{thermal} , t \in T \\
 & 1-COMMIT_{g,t} \geq \sum_{t'≥t-MinDown_g}^{t} SHUT_{g,t} &\forall \quad g \in G_{thermal} , t \in T \\
  & COMMIT_{g,t+1} - COMMIT_{g,t} = \quad START_{g,t+1} - SHUT_{g,t+1} &\forall \quad G_{thermal} \in G , t \in (1,...,T-1)\\
 &SOC_{t+1} = SOC_t + \big(CHARGE_{t+1} * battery\_eff - \frac{DISCHARGE_{t+1}}{battery\_eff}\big)  \quad &\forall t \in (1, ..., T-1)\\  
 &CHARGE_t \leq power\_cap \quad &\forall t \in T\\  
 &DISCHARGE_t \leq power\_cap \quad &\forall t \in T\\
 &SOC_t \leq energy\_cap \quad &\forall t \in T\\
 &SOC[0] = SOC[T] = 0.5 \times energy\_cap
\end{align}
$$


The **decision variables** in the above problem:

- $GEN_{g}$, generation (in MW) produced by each generator, $g$
- $START_{g,t}$, startup decision (binary) of thermal generator $g$ at time $t$
- $SHUT_{g,t}$, shutdown decision (binary) of thermal generator $g$ at time $t$
- $COMMIT_{g,t}$, commitment status (binary) of generator $g$ at time $t$
- $CHARGE_t$: How much DC power to charge to PHS (i.e. how much water to pump up water to upper reservoir); continuous
- $DISCHARGE_t$: How much DC power to discharge from PHS (i.e. how much water to release down to lower reservoir through turbines); continuous
- $SOC_t$: "Storage State of Charge" representing how much energy is stored in the battery (i.e., upper reservoir storage); continuous

The **parameters** are:

- $Pmin_g$, the minimum operating bounds for generator $g$ (based on engineering or natural resource constraints)
- $Pmax_g$, the maximum operating bounds for generator $g$ (based on engineering or natural resource constraints)
- $Demand$, the demand (in MW)
- $VarCost_g = VarOM_g + HeatRate_g \times FuelCost_g$, the variable cost of generator $g$
- $StartUpCost_g$, the startup cost of generator $g$
- $MinUp_g$, the minimum up time of generator $g$, or the minimum time after start-up before a unit can shut down
- $MinDown_g$, the minimum down time of generator $g$, or the minimum time after shut-down before a unit can start again
- $battery\_eff=0.84$: Fraction of the stored energy that can be converted to power.
- $power\_cap$: maximum power capacity of battery in MW, given in `existing_cap_mw` column of `gen_df` for `hydroelectric_pumped_storage`.
- $energy\_cap = 4 \times Pmax$: maximum energy in MWh that can be stored in the battery (i.e. upper reservoir)



$\color{red}{\text{Install gurobipy and load data on generators. The code below is complete.}}$

In [ ]:
!pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

# allow access to google drive
drive.mount('/content/drive')

datadir = "drive/MyDrive/Colab Notebooks/CE4110_6250/UCED_data/"
gen_info = pd.read_csv(datadir + "Generators_data.csv")
fuels = pd.read_csv(datadir + "Fuels_data.csv")
loads = pd.read_csv(datadir + "Demand.csv")
gen_variable = pd.read_csv(datadir + "Generators_capacity_factors.csv")

# Keep columns relevant to our UCED model
gen_info = gen_info.iloc[:,0:26] # first 26 columns
gen_df = gen_info.join(fuels.set_index("Fuel"), on="Fuel")
gen_df.rename(columns={"Cost_per_MMBtu": "Fuel_cost"},inplace=True)
gen_df["Fuel_cost"].fillna(0) # replace NAs for fuel cost with 0

# create "is_variable" column to indicate if this is a variable generation source (e.g. wind, solar):
gen_df["Is_variable"] = False
indices = [i for i in gen_df.index if gen_df["Resource"][i] in ["onshore_wind_turbine","small_hydroelectric","solar_photovoltaic"]]
gen_df["Is_variable"].iloc[indices] = True

# create full name of generator (including geographic location and cluster number)
#  for use with variable generation dataframe
gen_df["Gen_full"] = gen_df["region"] + "_" + gen_df["Resource"] + "_" + gen_df["cluster"].astype("str") + ".0"

# remove generators with no capacity (e.g. new build options that we'd use if this was capacity expansion problem)
gen_df = gen_df[gen_df["Existing_Cap_MW"] > 0]

# convert from GMT to GMT-8
gen_variable["Hour"] = np.mod(gen_variable["Hour"]-9, 8760)
gen_variable = gen_variable.sort_values(by="Hour")
loads["Hour"] = np.mod(loads["Hour"]-9, 8760)
loads = loads.sort_values(by="Hour")
loads.set_index("Hour",inplace=True)
gen_variable.set_index("Hour",inplace=True)
gen_variable

# convert gen_variables to H x G data frame of hourly capacity factors at each generator in gen_df
# this is so we can reference it by indices in the optimization that correspond to the indices in gen_df
capacity_factors = pd.DataFrame()
for source in gen_df["Gen_full"]:
  if source in gen_variable.columns.values:
    capacity_factors[source] = gen_variable[source]
  else:
    capacity_factors[source] = "NA"

# A spring day
n=100
T_period = np.arange(n*24,(n+1)*24)

# High solar case: 3,500 MW
gen_df_sens = gen_df.copy()
gen_df_sens["Existing_Cap_MW"][np.where(gen_df_sens["Resource"] == "solar_photovoltaic")[0]]= 3500

$\color{red}{\text{Extract the necessary PHS data from gen_df and then drop it from gen_df_sens and capacity_factors.}}$  
$ \color{red}{\text{The code below is complete.}}$

In [ ]:
# specify parameters to model PHS as a battery
power_cap = float(gen_df["Existing_Cap_MW"].iloc[np.where(gen_df["Resource"]=="hydroelectric_pumped_storage")])
energy_cap = 4 * power_cap
battery_eff = 0.84

# remove hydroelectric_pumped_storage from generators and capacity_factors dataframes
gen_df_sens.drop(index=gen_df.index[np.where(gen_df["Resource"]=="hydroelectric_pumped_storage")[0]],inplace=True)
gen_df_sens.reset_index(inplace=True)
gen_df_sens.drop(columns="index",inplace=True)

capacity_factors.drop(columns=["WEC_SDGE_hydroelectric_pumped_storage_1.0"],inplace=True)

$\color{red}{\text{Create the model, set the MIPGap and create indices for sets of variables. The code below only needs to be edited to include your license codes.}}$

In [ ]:
# Note we reduce the MIP gap tolerance threshold here to increase tractability
# Here we set it to a 1% gap (mip_gap=0.01), meaning that we will terminate once we have
# a feasible integer solution guaranteed to be within 1% of the objective
# function value of the optimal solution (e.g. the upper and lower bound are within 1% of
# each other as Gurobi traverses the branch and bound tree).
# Gurobi's default MIP gap is 0.0001 (0.01%), which can take a longer time for
# any complex problem. So it is important to set this to a realistic value.
mip_gap = 0.0001

# create environment with Gurobi academic license
# CHANGE THESE TO THE VALUES FOR YOUR LICENSE
params = {
"WLSACCESSID": ,
"WLSSECRET": ,
"LICENSEID": ,
}
env = gp.Env(params=params)

UCED = gp.Model('UCED model', env=env)
UCED.Params.MIPGap = mip_gap

# Define sets based on data
# Note the creation of several different sets of generators for use in
# different equations.
# Thermal resources for which unit commitment constraints apply
G_thermal = gen_df_sens.index[np.where(gen_df_sens["Up_time"] > 0)[0]]

# Non-thermal resources for which unit commitment constraints do NOT apply
G_nonthermal = gen_df_sens.index[np.where(gen_df_sens["Up_time"] == 0)[0]]

# Variable renewable resources
G_var = gen_df_sens.index[np.where(gen_df_sens["Is_variable"] == 1)[0]]

# Non-variable (dispatchable) resources
G_nonvar = gen_df_sens.index[np.where(gen_df_sens["Is_variable"] == 0)[0]]

# Non-variable and non-thermal resources
G_nonthermal_nonvar = np.intersect1d(G_nonvar, G_nonthermal)

# Set of all generators (above are all subsets of this)
G = gen_df_sens.index

# All time periods (hours) over which we are optimizing
H = loads.index[loads.index.isin(T_period)]

# A subset of time periods that excludes the last time period
H_red = H[0:-1]  # reduced time periods without last one

$\color{red}{\text{Edit the code below to include PHS decisions variables (and their upper bounds) and adapt demand constraint to include PHS discharge and charge with generation.}}$

\begin{align}
&CHARGE_t \leq power\_cap \quad &\forall t \in T\\  
&DISCHARGE_t \leq power\_cap \quad &\forall t \in T\\
&SOC_t \leq energy\_cap \quad &\forall t \in T\\
&\sum_{g \notin PHS} GEN_{g,t} + DISCHARGE_t - CHARGE_t = Demand_t \quad &\forall \quad t \in T\\
\end{align}

In [ ]:
# Decision variables
GEN = UCED.addVars(len(G), len(H), vtype=GRB.CONTINUOUS, name="GEN")
COMMIT = UCED.addVars(len(G_thermal), len(H), vtype=GRB.BINARY, name="COMMIT")
START = UCED.addVars(len(G_thermal), len(H), vtype=GRB.BINARY, name="START")
SHUT = UCED.addVars(len(G_thermal), len(H), vtype=GRB.BINARY, name = "SHUT")

# PHS decision variables (CHARGE, DISCHARGE and SOC)


# non-negativity constraints assumed, just add demand, capacity and unit commitment constraints
# demand constraint
c1_demand =

$\color{red}{\text{Thermal power constraints are unchanged. The code below is complete.}}$

In [ ]:
# capacity constraints
# 1. thermal generators requiring commitment
c2_min_thermal = UCED.addConstrs(GEN[g,h-H[0]] >= COMMIT[g-G_thermal[0],h-H[0]] * gen_df_sens["Existing_Cap_MW"].iloc[g] * gen_df_sens["Min_power"].iloc[g]
                                  for g in G_thermal
                                  for h in H)
c3_max_thermal = UCED.addConstrs(GEN[g,h-H[0]] <= COMMIT[g-G_thermal[0],h-H[0]] * gen_df_sens["Existing_Cap_MW"].iloc[g]
                                  for g in G_thermal
                                  for h in H)

# 2. non-variable generation not requiring commitment
c4_nonthermal_nonvar = UCED.addConstrs(GEN[g,h-H[0]] <= gen_df_sens["Existing_Cap_MW"].iloc[g]
                                  for g in G_nonthermal_nonvar
                                  for h in H)

# 3. variable generation, accounting for hourly capacity factor
c5_variable = UCED.addConstrs(GEN[g,h-H[0]] <= capacity_factors.iloc[h,g] * gen_df_sens["Existing_Cap_MW"].iloc[g]
                              for g in G_var
                              for h in H)

# unit commitment constraints
# 1. Minimum up time
c6_min_up_time = UCED.addConstrs(COMMIT[g-G_thermal[0],h-H[0]] >=
                                 gp.quicksum(START[g-G_thermal[0],hh-H[0]] for hh in np.intersect1d(H,range(h-gen_df_sens["Up_time"].iloc[g],h)))
                                  for g in G_thermal
                                 for h in H)

# 2. Minimum down time
c7_min_down_time = UCED.addConstrs(1-COMMIT[g-G_thermal[0],h-H[0]] >=
                                 gp.quicksum(SHUT[g-G_thermal[0],hh-H[0]] for hh in np.intersect1d(H, range(h-gen_df_sens["Down_time"].iloc[g],h)))
                                  for g in G_thermal
                                 for h in H)

# 3. Commitment state
c8_commitment_status = UCED.addConstrs(COMMIT[g-G_thermal[0],h-H[0]+1] - COMMIT[g-G_thermal[0],h-H[0]] ==
                                       START[g-G_thermal[0],h-H[0]+1] - SHUT[g-G_thermal[0],h-H[0]+1]
                                  for g in G_thermal
                                  for h in H_red)

$\color{red}{\text{Edit the code below to include SOC constraints for PHS.}}$  

\begin{align}
SOC_{t+1} = SOC_t + \big(CHARGE_{t+1} * battery\_eff - \frac{DISCHARGE_{t+1}}{battery\_eff}\big) \quad &\forall t \in (1, ..., T-1)\\
SOC[0] = SOC[T] = 0.5 \times energy\_cap
\end{align}

In [ ]:
# add PHS constraints
c9_SOC =
c10_SOC_init =
c11_SOC_final =

$\color{red}{\text{The variable O&M costs for PHS are 0, so the objective function code from class below is unchanged and complete.}}$

\begin{align}
\min &\sum_{g \notin PHS, t \in T} VarCost_g \times GEN_{g,t} + \sum_{g \in G_{thermal}, t \in T} StartUpCost_g \times START_{g,t} + \\
&\sum_{t \in T} VarCost_{PHS} \times (DISCHARGE_t + CHARGE_t)
\end{align}

In [ ]:
# Objective function
# Sum of variable costs + start-up costs for all generators and time periods
cost_nonvar_gen = gp.quicksum(
    gp.quicksum(GEN[g,h-H[0]] * (gen_df_sens["Var_OM_cost_per_MWh"].iloc[g] +
                                   gen_df_sens["Heat_rate_MMBTU_per_MWh"].iloc[g] * gen_df_sens["Fuel_cost"].iloc[g])
    for h in H)
  for g in G_nonvar
)

cost_var_gen = gp.quicksum(gp.quicksum(gen_df_sens["Var_OM_cost_per_MWh"].iloc[g] * GEN[g,h-H[0]]
    for g in G_var
    )
  for h in H
)

cost_startup = gp.quicksum(gen_df_sens["Start_cost_per_MW"].iloc[g] * gen_df_sens["Existing_Cap_MW"].iloc[g] * START[g-G_thermal[0],h-H[0]]
    for g in G_thermal
    for h in H)

UCED.setObjective(cost_nonvar_gen + cost_var_gen + cost_startup)

UCED.optimize()

$\color{red}{\text{Re-format the optimization results into data frames. The code below is complete.}}$

In [ ]:
rows = ["Hour" + str(h) for h in H]
commit = pd.DataFrame(columns=gen_df_sens["Gen_full"].loc[G_thermal], index=rows, data=0.0)
for h in H:
    for g in G_thermal:
        commit.loc["Hour"+str(h), gen_df_sens["Gen_full"].loc[g]] = COMMIT[g-G_thermal[0],h-H[0]].x

rows = ["Hour" + str(h) for h in H]
gen = pd.DataFrame(columns=gen_df_sens["Gen_full"], index=rows, data=0.0)
for h in H:
    for g in G:
        gen.loc["Hour"+str(h), gen_df_sens["Gen_full"].iloc[g]] = GEN[g,h-H[0]].x

rows = ["Hour" + str(h) for h in H]
phs = pd.DataFrame(columns=["Charge","Discharge","SOC"], index=rows, data=0.0)
for h in H:
  phs.loc["Hour"+str(h), "Charge"] = CHARGE[h-H[0]].x
  phs.loc["Hour"+str(h), "Discharge"] = DISCHARGE[h-H[0]].x
  phs.loc["Hour"+str(h), "SOC"] = SOC[h-H[0]].x

# calculate curtailment
curtail = np.zeros([len(H),len(G_var)])
for g in G_var:
  for h in H:
    curtail[h-H[0],g-G_var[0]] = capacity_factors.iloc[h,g] * gen_df_sens["Existing_Cap_MW"].iloc[g] - GEN[g,h-H[0]].x

curtail = pd.DataFrame(curtail, columns=capacity_factors.iloc[:,G_var].columns.values)
curtail.set_index(gen.index,inplace=True)

CCGT = [col for col in gen.columns.values if "combined" in col]
CT = [col for col in gen.columns.values if "combustion" in col]

btm = gen_variable["WEC_SDGE_solar_photovoltaic_1.0"].iloc[H]*600
btm = btm.to_frame()
btm.set_index(gen.index,inplace=True)

loads = loads.iloc[H].set_index(gen.index)

gen_by_type = pd.DataFrame({"Natural Gas CT": gen[CT].sum(axis=1), "Natural Gas CCGT": gen[CCGT].sum(axis=1),
                            "Biomass": gen['WEC_SDGE_biomass_1.0'], "Solar PV": gen['WEC_SDGE_solar_photovoltaic_1.0'],
                            "Small Hydro": gen['WEC_SDGE_small_hydroelectric_1.0'], "Onshore Wind": gen['WEC_SDGE_onshore_wind_turbine_1.0'],
                            "Curtailment": curtail.sum(axis=1), "Solar BTM": btm["WEC_SDGE_solar_photovoltaic_1.0"],
                            "PHS Charge": phs["Charge"], "PHS Discharge": phs["Discharge"], "SOC": phs["SOC"], "Demand": loads["Demand"]})
gen_by_type["Hour"] = gen_by_type.index.astype('string')

$\color{red}{\text{Write code below to make a stacked bar chart of the generation from each source, including PHS Charge and Discharge.}}$
$\color{red}{\text{For context, add a line for the demand, and on a twin axis, a line for SOC.}}$

$\color{red}{\text{Run the code below (same as from class) to see the number of thermal commitments each time step.}}$

In [ ]:
commit_by_type = pd.DataFrame({"Natural Gas CT": commit[CT].sum(axis=1), "Natural Gas CCGT": commit[CCGT].sum(axis=1)})
commit_by_type.plot.bar(stacked=True, width=1, color=["tab:orange","tab:blue"])